# Advanced Cipher Techniques

# 01. Hill Cipher Encryption and Decryption

## **Hill Cipher Overview**
The Hill Cipher is a polygraphic substitution cipher that uses linear algebra for encryption. It operates on blocks of letters and utilizes matrix multiplication for its computations.

### **Encryption Process**
1. **Key Matrix**: Choose a square key matrix (e.g., 2x2 or 3x3) with integer values. Ensure the determinant of the matrix is non-zero and coprime with 26 (for modular arithmetic).
2. **Plaintext Preparation**: Convert plaintext into numerical form (A=0, B=1, ..., Z=25). For a matrix of size NxN, pad the plaintext if its length is not a multiple of N.
3. **Matrix Multiplication**: Multiply the key matrix with the plaintext vector and take modulo 26 for each resulting element.
4. **Ciphertext Conversion**: Convert the resulting numerical values back to letters.

---

### **Encryption Dry Run**
#### Input:
- Plaintext: `ZIAURREHMAN`
- Key Matrix: 
   ```
  K = |  6  24 |
       |  1  13 |
  ```
- Modulo: 26

#### Steps:
1. Convert plaintext into numerical values (ignoring spaces):
   - Z = 25, I = 8, A = 0, U = 20, R = 17, R = 17, E = 4, H = 7, M = 12, A = 0, N = 13
   - Numerical form: `[25, 8, 0, 20, 17, 17, 4, 7, 12, 0, 13]`

2. Group into 2x1 column vectors (since the key matrix is 2x2):
   ```
   [25, 8], [0, 20], [17, 17], [4, 7], [12, 0], [13, 0]
   ```

3. Multiply each vector by the key matrix and take modulo 26:
   - For [25, 8]:
     ```
     C = (K * P) mod 26 = |  6  24 |   | 25 |
                          |  1  13 | * |  8 | mod 26 = |  4 |
                                                       | 25 |
     ```
     Result: `E, Z`
   - Repeat for all vectors.

4. Resulting Ciphertext:
   - Ciphertext: `EZMAQEKRUMGA`

---

### **Decryption Process**
1. **Key Matrix Inverse**: Calculate the inverse of the key matrix modulo 26. The inverse matrix \(K^{-1}\) satisfies:
   ```
   (K * K^{-1}) mod 26 = I
   ```
   Where \(I\) is the identity matrix.

2. **Ciphertext Preparation**: Convert the ciphertext into numerical form and group into column vectors.

3. **Matrix Multiplication**: Multiply the inverse key matrix by the ciphertext vector and take modulo 26.

4. **Plaintext Conversion**: Convert numerical values back to letters.

---

### **Python Implementation**

#### **Encryption Function:**

In [3]:
import numpy as np

def hill_cipher_encrypt(plaintext, key):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    plaintext = plaintext.replace(" ", "").upper()
    # Convert plaintext to numerical values
    plaintext_nums = [alphabet.index(char) for char in plaintext]

    # Pad plaintext if necessary
    n = key.shape[0]
    while len(plaintext_nums) % n != 0:
        plaintext_nums.append(alphabet.index('X'))  # Padding with 'X'

    # Reshape plaintext into column vectors
    plaintext_matrix = np.array(plaintext_nums).reshape(-1, n).T

    # Encrypt: Ciphertext = (Key * Plaintext) mod 26
    ciphertext_matrix = (np.dot(key, plaintext_matrix) % 26).T

    # Convert back to letters
    ciphertext = ''.join(alphabet[num] for row in ciphertext_matrix for num in row)
    return ciphertext

# Example
key = np.array([[6, 24], [1, 13]])
plaintext = "ZIAURREHMAN"
ciphertext = hill_cipher_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

Ciphertext: EZMAQEKRUMGA


# 02. Vigenère Cipher

## Introduction:
The Vigenère Cipher is a method of encrypting text by using a keyword to generate a repeating pattern of shifts. It extends the Caesar cipher by using multiple shifts based on the keyword's letters.

## Mathematical Steps:

1. Convert plaintext and keyword to numerical values.
2. Align the keyword with the plaintext by repeating it as necessary.
3. For each letter in the plaintext, shift it by the value of the corresponding keyword letter (modulo 26).

## Example:
**Plaintext:** "HELLO"  
**Keyword:** "KEY"

### Numerical Form:
- Plaintext: H = 7, E = 4, L = 11, L = 11, O = 14  
- Keyword: K = 10, E = 4, Y = 24 (repeat "KEY" if the plaintext is longer)

### Encryption Formula:
For each letter in the plaintext, we apply the following formula:



C_i = (P_i + K_i) % 26


Where:
- `C_i` is the ciphertext letter at position `i`
- `P_i` is the plaintext letter at position `i`
- `K_i` is the keyword letter at position `i`

### Dry Run:

- **Plaintext:** "HELLO"
- **Keyword:** "KEY"
- **Step 1:** Align the keyword with the plaintext by repeating it:

    ```
    Plaintext: H  E  L  L  O
    Keyword:   K  E  Y  K  E
    ```

- **Step 2:** Convert the letters to their corresponding numerical values (A=0, B=1, ..., Z=25):

    ```
    Plaintext: H=7, E=4, L=11, L=11, O=14
    Keyword:   K=10, E=4, Y=24, K=10, E=4
    ```

- **Step 3:** Apply the encryption formula `C_i = (P_i + K_i) % 26`:

    - For `H` and `K`:  
      `C_1 = (7 + 10) % 26 = 17` → `R`
    - For `E` and `E`:  
      `C_2 = (4 + 4) % 26 = 8` → `I`
    - For `L` and `Y`:  
      `C_3 = (11 + 24) % 26 = 9` → `J`
    - For `L` and `K`:  
      `C_4 = (11 + 10) % 26 = 21` → `V`
    - For `O` and `E`:  
      `C_5 = (14 + 4) % 26 = 18` → `S`

### Final Ciphertext:
**Ciphertext:** "RIJVS"


In [4]:
def vigenere_encrypt(plaintext, keyword):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    keyword_repeated = (keyword.upper() * (len(plaintext) // len(keyword) + 1))[:len(plaintext)]
    
    ciphertext = ''
    for p, k in zip(plaintext.upper(), keyword_repeated):
        shift = (alphabet.index(p) + alphabet.index(k)) % 26
        ciphertext += alphabet[shift]
    
    return ciphertext

# Example
plaintext = "HELLO"
keyword = "KEY"
ciphertext = vigenere_encrypt(plaintext, keyword)
print("Ciphertext:", ciphertext)


Ciphertext: RIJVS


# 03. Rail Fence Cipher

## Mathematical Description of Rail Fence Cipher:
The Rail Fence Cipher is a transposition cipher where the plaintext is written in a zigzag pattern across multiple rows (rails). Once the text is written in the zigzag pattern, the ciphertext is formed by reading the characters row by row.

### Encryption Process:
1. Choose the number of rails (rows), denoted as `n`.
2. Write the plaintext in a zigzag pattern across the rails.
3. Read the ciphertext by concatenating the characters row by row.

The number of rails determines how the characters are arranged in the zigzag pattern. For example, if the plaintext is written with 3 rails, the first row contains characters from the first and last parts of the zigzag, while the second row contains characters from the middle parts.

### Example (Plaintext: "HELLO", Rails: 3):

#### Step 1: Write the plaintext in a zigzag pattern across the rails.

- First, write the letters of the plaintext diagonally, as shown below:

| Rail 1 | H . . . O |
|--------|-----------|
| Rail 2 | . E . L . |
| Rail 3 | . . L . . |

#### Step 2: Read the ciphertext by concatenating the characters row by row.

- Read each row from left to right:
  - Rail 1: "HO"
  - Rail 2: "EL"
  - Rail 3: "L"

#### Step 3: Combine the rows to form the ciphertext:

**Ciphertext:** "HOELL"

---

## Python Code for Encryption:

In [7]:
# Example plaintext
plaintext = "HELLO"
rails = 3

# Encryption function
def rail_fence_encrypt(plaintext, rails):
    fence = [['' for _ in range(len(plaintext))] for _ in range(rails)]
    row, col = 0, 0
    direction = 1  # 1 means moving down, -1 means moving up

    for char in plaintext:
        fence[row][col] = char
        col += 1
        row += direction

        if row == 0 or row == rails - 1:
            direction *= -1

    ciphertext = ''.join(''.join(row) for row in fence)
    return ciphertext

# Encrypt the plaintext
ciphertext = rail_fence_encrypt(plaintext, rails)
print(f"Ciphertext: {ciphertext}")


Ciphertext: HOELL


# 04. One-Time Pad Cipher

## Mathematical Description of the One-Time Pad:
The **One-Time Pad (OTP)** is a symmetric-key encryption algorithm that uses a key that is as long as the plaintext. Each character of the plaintext is combined with the corresponding character of the key using modular addition. The key is used only once and then discarded, which is why it is called a "one-time" pad. 

### Encryption Process:
For each letter in the plaintext `P` and the corresponding letter in the key `K`, the ciphertext `C` is generated using the following formula:
\[
C_i = (P_i + K_i) \mod 26
\]
Where:
- \(P_i\) is the numerical value of the plaintext letter (0 for 'A', 1 for 'B', ..., 25 for 'Z'),
- \(K_i\) is the numerical value of the key letter (0 for 'A', 1 for 'B', ..., 25 for 'Z'),
- \(C_i\) is the resulting ciphertext letter.

### Decryption Process:
To decrypt the ciphertext, the inverse operation is applied:
\[
P_i = (C_i - K_i + 26) \mod 26
\]
This ensures that the plaintext can be recovered from the ciphertext using the same key.

### Example (Plaintext: "HELLO", Key: "XMCKL"):

1. **Convert the plaintext and key to numerical values**:
   - Plaintext "HELLO": H=7, E=4, L=11, L=11, O=14
   - Key "XMCKL": X=23, M=12, C=2, K=10, L=11

2. **Apply the encryption formula**:
   - \(C_1 = (P_1 + K_1) \mod 26 = (7 + 23) \mod 26 = 4\) → 'E'
   - \(C_2 = (P_2 + K_2) \mod 26 = (4 + 12) \mod 26 = 16\) → 'Q'
   - \(C_3 = (P_3 + K_3) \mod 26 = (11 + 2) \mod 26 = 13\) → 'N'
   - \(C_4 = (P_4 + K_4) \mod 26 = (11 + 10) \mod 26 = 21\) → 'V'
   - \(C_5 = (P_5 + K_5) \mod 26 = (14 + 11) \mod 26 = 25\) → 'Z'

3. **Resulting ciphertext**: "EQNVZ"

### Python Code for Encryption:

In [9]:
def otp_encrypt(plaintext, key):
    # Ensure that plaintext and key are the same length
    if len(plaintext) != len(key):
        raise ValueError("Plaintext and key must have the same length.")
    
    ciphertext = []
    for p_char, k_char in zip(plaintext, key):
        # Convert characters to numbers (A=0, B=1, ..., Z=25)
        p_val = ord(p_char) - ord('A')
        k_val = ord(k_char) - ord('A')
        
        # Apply the encryption formula
        c_val = (p_val + k_val) % 26
        
        # Convert back to character and append to the ciphertext
        ciphertext.append(chr(c_val + ord('A')))
    
    return ''.join(ciphertext)

# Example plaintext and key
plaintext = "HELLO"
key = "XMCKL"

# Encrypt the plaintext using the One-Time Pad Cipher
ciphertext = otp_encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext}")



Ciphertext: EQNVZ


# 05. Data Encryption Standards

## Data Encryption Standard (DES)

### Mathematical Understanding of DES:
The **Data Encryption Standard (DES)** is a symmetric-key algorithm for the encryption of electronic data. It operates on blocks of 64 bits using a 56-bit key. DES follows the Feistel structure, which involves multiple rounds of permutation and substitution. 

The DES algorithm works as follows:

1. **Initial Permutation (IP)**: The 64-bit plaintext is rearranged according to a predefined table called the initial permutation (IP).
   
2. **Rounds**: The data undergoes 16 rounds of processing. In each round:
   - The 64-bit data block is split into two 32-bit halves (L and R).
   - A round key (derived from the original key using a key schedule) is applied to the right half of the block.
   - A substitution-permutation (S-Box) process is applied to the right half of the data.
   - The result is combined with the left half using XOR and swapped for the next round.

3. **Final Permutation (FP)**: After 16 rounds, the data is subjected to a final permutation (FP) to get the ciphertext.

The inverse operations are used in decryption, with the round keys applied in reverse order.

### Encryption Process:
- Given a 64-bit plaintext \( P \) and a 56-bit key \( K \), DES generates 16 round keys.
- In each round, data is transformed using the key and S-box substitutions, followed by permutations.

### Python Encryption and Decryption (using `pycryptodome` library):

In [10]:
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

def des_encrypt(plaintext, key):
    cipher = DES.new(key, DES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(plaintext.encode(), DES.block_size))
    return cipher.iv + ciphertext

# Example usage
key = get_random_bytes(8)  # 56-bit key (8 bytes)
plaintext = "Hello DES!"
ciphertext = des_encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext.hex()}")

Ciphertext: cb909929d6979118e2684b2ff4e2119e6e15a86631899d5d


In [11]:
def des_decrypt(ciphertext, key):
    iv = ciphertext[:DES.block_size]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[DES.block_size:]), DES.block_size).decode()
    return plaintext

# Decrypting the previously encrypted message
decrypted_text = des_decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")


Decrypted Text: Hello DES!


# 06. Advanced Encryption Standard (AES)

Mathematical Understanding of AES:

The Advanced Encryption Standard (AES) is a symmetric-key algorithm that processes data in blocks of 128 bits using keys of 128, 192, or 256 bits. AES operates using several rounds (10, 12, or 14 rounds depending on the key size) that include substitution (S-box), permutation, and mixing steps.

The AES encryption process consists of four main stages in each round (except the last round):

- SubBytes: Each byte of the data block is substituted using an S-box.
- ShiftRows: Rows of the data are shifted by different offsets.
- MixColumns: The columns of the data matrix are mixed using a matrix multiplication.
- AddRoundKey: The round key is XORed with the data.
- The key expansion algorithm is used to generate the round keys from the original key.

### Encryption Process:

Given a 128-bit block of plaintext and a key (either 128, 192, or 256 bits), AES performs several rounds of substitution, permutation, and mixing operations, depending on the key length.

### Python Encryption and Decryption (using pycryptodome library):

In [12]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return cipher.iv + ciphertext

# Example usage
key = get_random_bytes(16)  # 128-bit key (16 bytes)
plaintext = "Hello AES!"
ciphertext = aes_encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext.hex()}")


Ciphertext: ed01ba1c4277992a022af23ec3f6b2471cb7d9e73861492bc1dc9c4c041214f9


In [13]:
from Crypto.Util.Padding import unpad

def aes_decrypt(ciphertext, key):
    iv = ciphertext[:AES.block_size]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[AES.block_size:]), AES.block_size).decode()
    return plaintext

# Decrypting the previously encrypted message
decrypted_text = aes_decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")


Decrypted Text: Hello AES!
